# Day 12

In [43]:
import requests
from bs4 import BeautifulSoup

def get_aoc_problem(day, year=2023):
    url = f"https://adventofcode.com/{year}/day/{day}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # raises an exception for HTTP errors

        soup = BeautifulSoup(response.text, 'html.parser')
        
        problem_text = soup.find('article').get_text()
        return problem_text
    except Exception as e:
        return f"Error fetching problem: {e}"

day = 12
problem_prompt = get_aoc_problem(day)
print(problem_prompt)

--- Day 12: Hot Springs ---You finally reach the hot springs! You can see steam rising from secluded areas attached to the primary, ornate building.
As you turn to enter, the researcher stops you. "Wait - I thought you were looking for the hot springs, weren't you?" You indicate that this definitely looks like hot springs to you.
"Oh, sorry, common mistake! This is actually the onsen! The hot springs are next door."
You look in the direction the researcher is pointing and suddenly notice the massive metal helixes towering overhead. "This way!"
It only takes you a few more steps to reach the main gate of the massive fenced-off area containing the springs. You go through the gate and into a small administrative building.
"Hello! What brings you to the hot springs today? Sorry they're not very hot right now; we're having a lava shortage at the moment." You ask about the missing machine parts for Desert Island.
"Oh, all of Gear Island is currently offline! Nothing is being manufactured at 

In [44]:
try:
    # Open and read the file
    with open('input.txt', 'r') as file:
        lines = file.read().strip().split('\n')

    # Print each line
    for line in lines:
        print(line)

except FileNotFoundError:
    # Specific exception for a clearer error message
    print('Input file not found.')

except Exception as e:
    # Catch other exceptions and print the error
    print(f'An error occurred: {e}')


..???.??.? 1,1,1
?#?##???.???? 2,5,1,1
?#??????##? 1,1,2
?#.#?#??#??? 1,7,1
?#???#?#??.#.###.? 3,1,3,1,3,1
?#......#.?.?. 1,1,1
.????#????? 2,1,2
#.?...?#????#??? 1,7,1
?.#??.??#? 2,1,1
.#???###?#??#???#?.? 7,1,7
?###?.??#??..? 5,4
.#?.??#????#?.?. 2,4,1,1
#.#??????? 1,1,4
#????##.???#???. 1,1,2,1,5
?.?.##???#???##.? 2,7
????.?#?.? 2,2
??.#?????#??#. 1,10
???????.???#?? 6,1,2
#?.#???.?? 2,4,1
?####????.?.?? 4,3,2
?????#???? 6,1
????.???????#?#.? 1,5
.#???##.?#??#??? 6,5
??#?#?????????????. 8,4,1
???.????#?#.???? 7,4
?#???#?.?????????? 2,1,8,1
#?.#?#?.?? 1,3,2
?.??##??????.?. 1,5,1
????#????????#????.? 5,9
?.#?#????? 1,5
??????#.??.??.? 1,5,1,2
##?#??????#.#?#??? 5,1,2,6
????.?##?#..# 2,3,1,1
??.?..???.?#?#? 2,4
???##???.?#??#?# 1,5,6
??.????#?????#?? 1,9
??##?#????.#???.#??? 1,8,4,1,1
##?#???????. 4,1,3
?#???##?????#? 1,1,6,1
????????##?#?????? 1,14
????#????.??? 1,2,1,1
?##.???.#???.#?.??# 3,1,1,4,2,1
?##???????? 2,2,1
?????#??????.??# 1,5,1,1
#?#?????????#.? 5,1,1,2,1
.?.?.??.#. 1,1,

### Utility functions

In [77]:
def process_line(line, part_2 = False):
    '''
    input : string (line of the input)

    output : int (number of possible different configurations for that line)
    '''

    row_data, digit_data = line.split()[0], line.split()[1]

    index_list = []

    for i, row_char in enumerate(row_data):
        if row_char == '?':
            index_list.append(i)

    if not part_2:
        num_of_configurations = check_configurations(row_data, index_list, 0, digit_data)
    else:
        print('PART 2')
        num_of_configurations = check_configurations_2(row_data, index_list, 0, digit_data)

    return num_of_configurations



In [46]:
def check_configurations(row_data, index_list, current_index, digit_data):
    # print(f'Called check_configurations \n with: (row_data, index_list, current_index, digit_data) = {(row_data, index_list, current_index, digit_data)}')
    
    if current_index < len(index_list):
        # havent reached the end yet:

        index_to_change = index_list[current_index]

        # check for errors
        assert row_data[index_to_change] == '?'

        if index_to_change > 0:
            prefix = row_data[:index_to_change]
        else: 
            prefix = ""
        
        if index_to_change+1 < len(row_data):
            suffix = row_data[index_to_change+1:]
        else:
            suffix = ""
        
        row_data_A = prefix + '.' + suffix
        row_data_B = prefix + '#' + suffix

        configuration_count = 0

        configuration_count += check_configurations(row_data_A, index_list, current_index + 1, digit_data)
        configuration_count += check_configurations(row_data_B, index_list, current_index + 1, digit_data)

        return configuration_count

    else:
        # reached the end, time to validate:

        # should have no unknown spots at this point
        assert '?' not in row_data

        return 1 if get_digit_data(row_data) == digit_data else 0

In [47]:
def get_digit_data(row_str):
    current_length = 0
    out_str_nums = []
    for i in range(len(row_str)):
        if row_str[i] == '#':
            current_length += 1
        elif current_length > 0:
            out_str_nums.append(str(current_length))
            current_length = 0
    
    if current_length > 0:
            out_str_nums.append(str(current_length))
      
    return ','.join(out_str_nums)

### Testing get_digit_data

In [48]:
test_lines = [
    "#.#.###",
    ".#...#....###.",
    ".#.###.#.######",
    "####.#...#...",
    "#....######..#####.",
    ".###.##....#"
]

for test_line in test_lines:
    print(get_digit_data(test_line))

1,1,3
1,1,3
1,3,1,6
4,1,1
1,6,5
3,2,1


In [80]:
def get_partial_digit_data(row_str):
    current_length = 0
    out_str_nums = []

    #everything to the left of the ?
    left_part = row_str.split("?")[0]

    for i in range(len(left_part)):
        if left_part[i] == '#':
            current_length += 1
        elif current_length > 0:
            out_str_nums.append(str(current_length))
            current_length = 0
    
    if current_length > 0:
            out_str_nums.append(str(current_length))
      
    return ','.join(out_str_nums)

In [81]:
def check_configurations_2(row_data, index_list, current_index, digit_data):
    # print(f'Called check_configurations \n with: (row_data, index_list, current_index, digit_data) = {(row_data, index_list, current_index, digit_data)}')
    
    if current_index < len(index_list):
        # havent reached the end yet:

        index_to_change = index_list[current_index]

        # check for errors
        assert row_data[index_to_change] == '?'

        if index_to_change > 0:
            prefix = row_data[:index_to_change]
        else: 
            prefix = ""
        
        if index_to_change+1 < len(row_data):
            suffix = row_data[index_to_change+1:]
        else:
            suffix = ""
        
        row_data_A = prefix + '.' + suffix
        row_data_B = prefix + '#' + suffix

        configuration_count = 0

        # check if the current row_data is valid:
        if validate_incomplete_row(row_data_A, digit_data):
            configuration_count += check_configurations_2(row_data_A, index_list, current_index + 1, digit_data)

        if validate_incomplete_row(row_data_B, digit_data):
            configuration_count += check_configurations_2(row_data_B, index_list, current_index + 1, digit_data)

        return configuration_count

    else:
        # reached the end, time to validate:

        # should have no unknown spots at this point
        assert '?' not in row_data

        return 1 if get_digit_data(row_data) == digit_data else 0

In [90]:
def validate_incomplete_row(row_data, digit_data):
    '''returns true if the current row_data is possible given the digit_data'''

    partial_digit_data = get_partial_digit_data(row_data)

    digit_list = digit_data.split(',')
    partial_digit_list = partial_digit_data.split(',')

    print(digit_list, partial_digit_list)

    for i in range(len(partial_digit_list)):
        if i > len(digit_list):
            return False
        if partial_digit_list[i] == digit_list[i]:
            continue
        elif int(partial_digit_list[i]) > int(digit_list[i]):
            return False
    
    return True
    

In [91]:
print(validate_incomplete_row('.#.###.#.######','1,3,1,6'))
print(validate_incomplete_row('.#.###.#.##.###','1,3,1,6'))

['1', '3', '1', '6'] ['1', '3', '1', '6']
True
['1', '3', '1', '6'] ['1', '3', '1', '2', '3']


IndexError: list index out of range

### Part 1

In [50]:
from tqdm import tqdm

total_sum = 0

for line in tqdm(lines):
    total_sum += process_line(line)

print(total_sum)

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:16<00:00, 62.46it/s]

8419


### Part 2

In [82]:
def part_2_line_process(line):
    left_line, right_line = line.split()[0], line.split()[1]

    left_og = left_line
    right_og = right_line

    for _ in range(4):
        left_line += '?' + left_og

    for _ in range(4):
        right_line += ',' + right_og

    return left_line + ' ' + right_line

In [83]:
total_new_sum = 0

for line in tqdm(lines):
    new_line = part_2_line_process(line)
    total_new_sum += process_line(new_line, part_2 = True)

print(f'Total sum of configurations = {total_new_sum}')
    

100%|██████████| 1000/1000 [00:00<00:00, 37567.88it/s]

PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2
PART 2

In [75]:
test_row_1 = '?#.###.#.######'
test_row_2 = '.#.###.?.######'
test_row_3 = '.#.###.#.##?###'


print(get_partial_digit_data(test_row_1))
print(get_partial_digit_data(test_row_2))
print(get_partial_digit_data(test_row_3))


1,3
1,3,1,2
